# NLP - CEIA - 2024

## Entrega 1


### Autor: Julio Donadello
### Email: juliodonadello@gmail.com

- Nota: Se dejo el codigo previo al desafio como funete de consulta.

---

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[1]

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

In [21]:
# en el atributo `data` accedemos al texto
newsgroups_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [9]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [11]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [12]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [15]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [22]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [23]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [24]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [25]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [26]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [27]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [35]:
mostsim

array([6635, 4253, 3596, 4271, 3746], dtype=int64)

In [41]:
print(newsgroups_train.data[3746])

THE WHITE HOUSE

                  Office of the Press Secretary
                 (Vancouver, British Columbia) 
______________________________________________________________


                       BACKGROUND BRIEFING
                               BY
                 SENIOR ADMINISTRATION OFFICIALS


                          April 4, 1993
	     
                          Canada Place
                  Vancouver, British Columbia  


9:40 A.M. PST
	     
	     
	     Folks, we're about to start the BACKGROUND BRIEFING 
on the aid package.

	     SENIOR ADMINISTRATION OFFICIAL:  Good morning.  The 
President -- President Clinton and President Yeltsin agreed 
yesterday on a series of American initiatives to support economic 
and political reform in Russia, and it's valued at $1.6 billion.  

	     Before taking your questions and running through the 
basic outlines of this package, I want to make a few points.  
First, this is the maximum that the Clinton administration can do 
with 

In [28]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [42]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [44]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [48]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

---

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


---

### 1. Vectorizar

Se va a trabajar con el subset de train

In [63]:
import random

In [94]:
# seleccion documentos random
idxs = random.sample(range(len(newsgroups_train.data)), 5)

In [99]:
for idx in idxs:
    # Calcular la similaridad del coseno con todos los documentos
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    # Obtener los 5 documentos más similares
    most_similar = np.argsort(cossim)[::-1][1:6]

    # Mostrar el documento original y los 5 más similares
    print(f"Documento original: {newsgroups_train.data[idx][0:100]}")
    print("-" * 50)
    print(f"Clase original: {newsgroups_train.target_names[y_train[idx]]}")
    print("-" * 50)
    print("Documentos similares:")
    for i in most_similar:
        print(f" - {newsgroups_train.data[i][0:100]}")
        print("-" * 50)
        print(f"   Clase: {newsgroups_train.target_names[y_train[i]]}")
        print("-" * 50)
    print("-" * 50)

Documento original: 
She hasn't.  Dorothy Denning has spent many years earning the
professional respect of her colleague
--------------------------------------------------
Clase original: sci.crypt
--------------------------------------------------
Documentos similares:
 - *******
*******  This is somewhat long, but pleas read it!!!!!!!!!!!!!!!!!
*******



Boy am i glad 
--------------------------------------------------
   Clase: soc.religion.christian
--------------------------------------------------
 - Being a parent in need of some help, I ask that you bear with me while I
describe the situation whic
--------------------------------------------------
   Clase: soc.religion.christian
--------------------------------------------------
 - :If the Clinton Clipper is so very good, why not make its algrithm public
:so many people can exchan
--------------------------------------------------
   Clase: sci.crypt
--------------------------------------------------
 - :    Does Dorothy Denn

- Se observan categorias similares en el top 5 de cosine similarity.

### 2. Entrenar modelos Naive Bayes

Se van a utilizar los siguientes parametros:

- Vectorizador:
    - max_df: Ignora términos que aparecen en más del porcentaje especificado de documentos (0.4). 
    - min_df: Ignora términos que aparecen en menos del porcentaje especificado de documentos (6).
    - ngram_range: Considera unigramas y bigramas (1,2).
    - stop_words: Elimina palabras comunes que no aportan mucha información ('english'>  "the", "and", "is", "in").

- Modelo:
    - alpha: Parámetro de suavizado para Naive Bayes.

In [158]:
tfidfvect = TfidfVectorizer(max_df=0.4, min_df=6, ngram_range=(1,2), stop_words='english')
X_train = tfidfvect.fit_transform(newsgroups_train.data)
X_test = tfidfvect.transform(newsgroups_test.data)

In [132]:
# Ejemplo para el modelo MultinomialNB
clf = MultinomialNB(alpha=0.0999)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f1_score(y_test, y_pred, average='macro'))

0.6754589155731984


In [119]:
# Ejemplo para el modelo ComplementNB
clf2 = ComplementNB(alpha=0.999)
clf2.fit(X_train, y_train)
y_pred = clf2.predict(X_test)
print(f1_score(y_test, y_pred, average='macro'))

0.686676146091848


- Se observa una mejora en los valores de f1 para los datos de test al utilizar los parametros respecto del modelo base implementado por el profesor (f1 = 0.58).

### 3. Transponer la matriz documento-término

Se va a trabajar con el subset de train.

In [159]:
# Se rearma el dict idx2word
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [160]:
# Se debe transponer la matriz documento-término para obtener la matriz término-documento

X_train_t = X_train.T

In [161]:
print(X_train.shape) # doc-term
print(X_train_t.shape) # term-doc

(11314, 22863)
(22863, 11314)


In [163]:
words = ['baseball', 'body', 'religion', 'accidents', 'new york']

for word in words:
    # Obtener el índice de la palabra
    idx = tfidfvect.vocabulary_[word]

    # Calcular la similaridad del coseno con todas las palabras
    cossim = cosine_similarity(X_train_t[idx], X_train_t)[0]

    # Obtener las 5 palabras más similares
    most_similar = np.argsort(cossim)[::-1][1:6]

    # Mostrar la palabra original y las 5 más similares
    print(f"Palabra original: {word}")
    print("Palabras similares:")
    for i in most_similar:
        print(f" - {idx2word[i]}")
    print("-" * 50)

Palabra original: baseball
Palabras similares:
 - baseball game
 - league baseball
 - football
 - tommorrow
 - espn
--------------------------------------------------
Palabra original: body
Palabras similares:
 - supplement
 - body know
 - resurrection
 - essential
 - favorite
--------------------------------------------------
Palabra original: religion
Palabras similares:
 - religion does
 - religious
 - freedom religion
 - religions
 - categorized
--------------------------------------------------
Palabra original: accidents
Palabras similares:
 - hammer
 - suicides
 - licence
 - safest
 - soviet union
--------------------------------------------------
Palabra original: new york
Palabras similares:
 - york
 - york times
 - toronto blue
 - york city
 - new
--------------------------------------------------


- Las similaridades entre palabras tienen mucho sentido.